In [ ]:
%fs ls dbfs:/FileStore/tables/nfl-big-data-bowl-2021/

In [ ]:
# Read data from csv files
nfl_folder = dbutils.fs.ls("dbfs:/FileStore/tables/nfl-big-data-bowl-2021/")
files = [file[0] for file in nfl_folder]

games = spark.read.format('csv').option('inferSchema', True).option(
    'header', True).option('sep', ',').load(
        files.pop(
            files.index(
                'dbfs:/FileStore/tables/nfl-big-data-bowl-2021/games.csv')))
players = spark.read.format('csv').option('inferSchema', True).option(
    'header', True).option('sep', ',').load(
        files.pop(
            files.index(
                'dbfs:/FileStore/tables/nfl-big-data-bowl-2021/players.csv')))
plays = spark.read.format('csv').option('inferSchema', True).option(
    'header', True).option('sep', ',').load(
        files.pop(
            files.index(
                'dbfs:/FileStore/tables/nfl-big-data-bowl-2021/plays.csv')))
weeks = [
    spark.read.format('csv').option('inferSchema',
                                    True).option('header',
                                                 True).option('sep',
                                                              ',').load(file)
    for file in files
]

In [ ]:
# Tests to make sure all the files were uploaded correctly
assert len(weeks) == 17, 'Number of weeks should be 17'
assert len(nfl_folder) == 20, 'Number of files should be 20'

In [ ]:
# Merging all 17 tracking csv files into one DataFrame then sorting by Time
tracking = weeks[0]
for week in range(1, len(weeks)):
    tracking = tracking.union(weeks[week])
tracking = tracking.sort('time')

In [ ]:
display(tracking)

In [ ]:
tracking.count()
assert tracking.count(
) == 18309388, 'Number of rows in tracking df should be 18309388'

In [ ]:
# Save all 4 dataframes as tables for quicker reads
games.write.saveAsTable('nfl_games')
players.write.saveAsTable('nfl_players')
plays.write.saveAsTable('nfl_plays')
tracking.write.saveAsTable('nfl_tracking')

In [ ]:
%fs ls dbfs:/user/hive/warehouse